# Start

In [1]:
import importlib
import os
import sys
import csv
import datetime 
import json
import importlib.util
import inspect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
from pathlib import Path
from docx import Document
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.shared import Inches
from typing import Optional

from langchain_core.tools import Tool, StructuredTool
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage

from bsm_multi_agents.graph.state import WorkflowState

In [2]:
cwd = Path.cwd()
project_path = cwd.parent.parent
project_path

PosixPath('/Users/yifanli/Github/model_doc_automation/TooTwo_mcp')

In [3]:
DEFAULT_SCENARIOS = [
    {"name": "Black Monday (1987)", "spot_change": -0.20, "vol_change": 0.50, "rate_change": -0.005},
    {"name": "Dot-com Crash (2000)", "spot_change": -0.70, "vol_change": 2.00, "rate_change": -0.02},
    {"name": "2008 Financial Crisis", "spot_change": -0.50, "vol_change": 1.50, "rate_change": -0.01},
    {"name": "VIX Spike (No Stock Move)", "spot_change": 0.0, "vol_change": 1.00, "rate_change": 0.0},
    {"name": "Rate Shock (+200bps)", "spot_change": 0.0, "vol_change": 0.0, "rate_change": 0.02},
    {"name": "Liquidation Scenario", "spot_change": -0.30, "vol_change": 3.00, "rate_change": 0.01},
    {"name": "Volatility Collapse", "spot_change": 0.05, "vol_change": -0.50, "rate_change": 0.0},
]

# MCP Server

## call_mcp_tool_async

In [19]:
from bsm_multi_agents.agents.mcp_adapter import call_mcp_tool

In [20]:
server_script_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
tool_name = "calculate_bsm_to_file"
args = {
    "input_path": csv_file_path,
    "output_dir": output_dir,
}

In [21]:
call_mcp_tool(tool_name, server_script_path, args)

CallToolResult(meta=None, content=[TextContent(type='text', text='Unknown tool: calculate_bsm_to_file', annotations=None, meta=None)], structuredContent=None, isError=True)

## list_mcp_tools_sync

In [22]:
from bsm_multi_agents.agents.mcp_adapter import list_mcp_tools_sync

In [23]:
server_script_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")

In [24]:
list_mcp_tools_sync(server_script_path)

[Tool(name='calculate_greeks_to_file', title=None, description='\n    Reads the CSV at input_path, calculates greeks, and saves the result to output_dir.\n    Returns the path to the result file.\n    ', inputSchema={'properties': {'input_path': {'title': 'Input Path', 'type': 'string'}, 'output_dir': {'default': './output', 'title': 'Output Dir', 'type': 'string'}}, 'required': ['input_path'], 'title': 'calculate_greeks_to_fileArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': 'calculate_greeks_to_fileOutput', 'type': 'object'}, icons=None, annotations=None, meta=None),
 Tool(name='validate_greeks_to_file', title=None, description='\n    Validate Greeks for ALL options from CSV data.\n\n    For each option:\n    - Validates: BSM_price > 0\n    - Validates: delta in [0,1] for calls, [-1,0] for puts\n    - Validates: gamma >= 0, vega >= 0\n\n    Args:\n        state: InjectedState, state from the

## mcp_tool_to_langchain_tool

In [25]:
from bsm_multi_agents.agents.mcp_adapter import list_mcp_tools_sync
server_script_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
mcp_tools = list_mcp_tools_sync(server_script_path)
mcp_tool = mcp_tools[0]
print(mcp_tool)

name='calculate_greeks_to_file' title=None description='\n    Reads the CSV at input_path, calculates greeks, and saves the result to output_dir.\n    Returns the path to the result file.\n    ' inputSchema={'properties': {'input_path': {'title': 'Input Path', 'type': 'string'}, 'output_dir': {'default': './output', 'title': 'Output Dir', 'type': 'string'}}, 'required': ['input_path'], 'title': 'calculate_greeks_to_fileArguments', 'type': 'object'} outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': 'calculate_greeks_to_fileOutput', 'type': 'object'} icons=None annotations=None meta=None


In [26]:
from bsm_multi_agents.agents.mcp_adapter import mcp_tool_to_langchain_tool

In [27]:
test_tool = mcp_tool_to_langchain_tool(mcp_tool, server_script_path)

In [28]:
test_tool.invoke({"input_path": csv_file_path, "output_dir": output_dir})

CallToolResult(meta=None, content=[TextContent(type='text', text='/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results.csv', annotations=None, meta=None)], structuredContent={'result': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results.csv'}, isError=False)

# Local Tools

## load_local_tools_from_file

In [35]:
file_path = os.path.join(project_path, "src/bsm_multi_agents/tools/my_add.py")

In [38]:
module_name = os.path.basename(file_path).replace(".py", "")
spec = importlib.util.spec_from_file_location(module_name, file_path)

module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)

tools = []
for name, func in inspect.getmembers(module, inspect.isfunction):
    # Exclude private functions or imports
    if not name.startswith("_") and func.__module__ == module_name:
        tools.append(StructuredTool.from_function(func))
tools
    

[StructuredTool(name='my_add', description='Add two integers.\n\nArgs:\n    a (int): The first integer.\n    b (int): The second integer.\n\nReturns:\n    int: The sum of the two integers.', args_schema=<class 'langchain_core.utils.pydantic.my_add'>, func=<function my_add at 0x11a701f80>)]

## load_local_tools_from_folder

In [41]:
from bsm_multi_agents.agents.utils import load_local_tools_from_file

In [42]:
folder_path = os.path.join(project_path, "src/bsm_multi_agents/tools")

In [44]:
tools = []
for file_name in os.listdir(folder_path):
    if file_name.endswith(".py") and file_name[0] not in "._":
        file_path = os.path.join(folder_path, file_name)
        tools.extend(load_local_tools_from_file(file_path))
tools

[StructuredTool(name='my_add', description='Add two integers.\n\nArgs:\n    a (int): The first integer.\n    b (int): The second integer.\n\nReturns:\n    int: The sum of the two integers.', args_schema=<class 'langchain_core.utils.pydantic.my_add'>, func=<function my_add at 0x11a7f2160>)]

## load_tools_from_mcp_and_local

# Tools

## calculate_bsm_to_file

In [34]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [18]:
from bsm_multi_agents.mcp import pricing_calculator
importlib.reload(pricing_calculator)
from bsm_multi_agents.mcp.pricing_calculator import calculate_bsm_to_file

In [19]:
calculate_bsm_to_file(csv_file_path, output_dir)

/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/pricing_calculator.py:11: RuntimeWarning: invalid value encountered in sqrt
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/pricing_calculator.py:12: RuntimeWarning: invalid value encountered in sqrt
  d2 = d1 - sigma * np.sqrt(T)
/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/pricing_calculator.py:11: RuntimeWarning: invalid value encountered in log
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))


'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv'

## calculate_greeks_to_file

In [20]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [21]:
from bsm_multi_agents.mcp import pricing_calculator
importlib.reload(pricing_calculator)
from bsm_multi_agents.mcp.pricing_calculator import calculate_greeks_to_file

In [22]:
calculate_greeks_to_file(csv_file_path, output_dir)

'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results.csv'

## validate_greeks_to_file

In [106]:
greeks_results_path = os.path.join(project_path, "data/cache/dummy_options_greeks_results.csv")
output_dir = os.path.join(project_path, "data/cache")

In [107]:
from bsm_multi_agents.mcp import pricing_validator
importlib.reload(pricing_validator)
from bsm_multi_agents.mcp.pricing_validator import validate_greeks_to_file

In [108]:
validate_greeks_to_file(greeks_results_path, output_dir)

'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results_validate_results.csv'

## _run_stress_test

In [3]:
from bsm_multi_agents.mcp.pricing_validator import (
    _bsm_price
)

In [4]:
input_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [5]:
df = pd.read_csv(input_path)
scenario = {"name": "Black Monday (1987)", "spot_change": -0.20, "vol_change": 0.50, "rate_change": -0.005}

In [6]:
name = scenario.get('name', 'Unknown')
spot_change = float(scenario.get('spot_change', 0.0))
vol_change = float(scenario.get('vol_change', 0.0))
rate_change = float(scenario.get('rate_change', 0.0))
opt_type = df['option_type']
S = df['S']
K = df['K']
T = df['T']
r = df['r']
sigma = df['sigma']
base_price = _bsm_price(opt_type, S, K, T, r, sigma)

/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
S_stressed = S * (1 + spot_change)
sigma_stressed = np.maximum(0.001, sigma + vol_change)
r_stressed = r + rate_change
stressed_price = _bsm_price(opt_type, S_stressed, K, T, r_stressed, sigma_stressed)

/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
pnl = stressed_price - base_price
pnl_pct = np.where(base_price != 0, (pnl / base_price * 100), 0.0)
df["base_price"] = base_price
df[f"{name}_spot_change_pct"] = spot_change * 100
df[f"{name}_vol_change_pct"] = vol_change * 100
df[f"{name}_rate_change_pct"] = rate_change * 100
df[f"{name}_stressed_price"] = stressed_price
df[f"{name}_PnL"] = pnl
df[f"{name}_PnL%"] = pnl_pct
df

/var/folders/r_/js4cr3ps49j4qkf0dy2w4lm00000gn/T/ipykernel_21662/3535194378.py:2: RuntimeWarning: invalid value encountered in divide
  pnl_pct = np.where(base_price != 0, (pnl / base_price * 100), 0.0)


,date,S,K,T,r,sigma,option_type,asset_class,scenario_name,spot_change_pct,vol_change_pct,rate_change_pct,Black Monday (1987)_PnL,Black Monday (1987)_PnL%
0,2025-09-01,1.124836,1.147031,0.396435,0.016240,2.500000,call,FX,Black Monday (1987),-20.0,50.0,-0.5,-0.084953,-13.345616
1,2025-09-02,98.617357,105.839622,1.242119,0.038323,0.164521,call,Equity,Black Monday (1987),-20.0,50.0,-0.5,9.961039,160.811947
2,2025-09-03,95.305256,89.270613,0.678060,0.030990,0.195456,call,Equity,Black Monday (1987),-20.0,50.0,-0.5,2.648833,24.923342
3,2025-09-04,105.425600,101.023642,-0.100000,0.015580,0.257986,call,Equity,Black Monday (1987),-20.0,50.0,-0.5,-4.401959,-100.000000
4,2025-09-05,1.054599,1.114748,0.479380,0.030569,0.095607,call,FX,Black Monday (1987),-20.0,50.0,-0.5,0.049762,425.540282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2025-09-16,55.265764,59.074121,0.706291,0.043157,2.500000,put,Commodity,Black Monday (1987),-20.0,50.0,-0.5,6.291021,15.424250
196,2025-09-17,100.961208,93.299603,0.246211,0.037852,0.207502,put,Equity,Black Monday (1987),-20.0,50.0,-0.5,17.770774,1646.594583
197,2025-09-18,95.377247,99.664774,0.224177,0.022612,0.234969,put,Equity,Black Monday (1987),-20.0,50.0,-0.5,20.398273,315.396223
198,2025-09-19,1.172349,1.201867,1.783358,0.034635,0.081875,put,FX,Black Monday (1987),-20.0,50.0,-0.5,0.392761,1235.920504


## run_stress_test_to_file

In [9]:
from bsm_multi_agents.mcp.pricing_validator import (
    _bsm_price,
    _run_stress_test,
    # run_stress_test_to_file,
    DEFAULT_SCENARIOS
)

In [5]:
input_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [6]:
df = pd.read_csv(input_path)
stress_scenarios = DEFAULT_SCENARIOS

In [7]:
df = df.drop_duplicates().reset_index(drop=True)
required_cols = ['option_type', 'S', 'K', 'T', 'r', 'sigma']
missing = [c for c in required_cols if c not in df.columns]
missing

[]

In [13]:
df['base_price'] = _bsm_price(df['option_type'], df['S'], df['K'], df['T'], df['r'], df['sigma'])
pnl_cols = []
scenario_results = []
for scenario in stress_scenarios:
    res_df = _run_stress_test(df, scenario)
    scenario_results.append(res_df)
    pnl_cols.append(f"{scenario['name']}_PnL")
final_df = pd.concat([df] + scenario_results, axis=1)
final_df

/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:39

,date,S,K,T,r,sigma,option_type,asset_class,base_price,Black Monday (1987)_spot_change_pct,...,Liquidation Scenario_rate_change_pct,Liquidation Scenario_stressed_price,Liquidation Scenario_PnL,Liquidation Scenario_PnL%,Volatility Collapse_spot_change_pct,Volatility Collapse_vol_change_pct,Volatility Collapse_rate_change_pct,Volatility Collapse_stressed_price,Volatility Collapse_PnL,Volatility Collapse_PnL%
0,2025-09-01,1.124836,1.147031,0.396435,0.016240,2.500000,call,FX,0.636560,-20.0,...,1.0,0.708879,0.072319,11.360985,5.0,-50.0,0.0,5.675062e-01,-0.069053,-10.847902
1,2025-09-02,98.617357,105.839622,1.242119,0.038323,0.164521,call,Equity,6.194216,-20.0,...,1.0,62.601166,56.406950,910.639075,5.0,-50.0,0.0,2.628706e+00,-3.565510,-57.561928
2,2025-09-03,95.305256,89.270613,0.678060,0.030990,0.195456,call,Equity,10.627919,-20.0,...,1.0,52.427188,41.799268,393.296812,5.0,-50.0,0.0,1.265620e+01,2.028281,19.084462
3,2025-09-04,105.425600,101.023642,-0.100000,0.015580,0.257986,call,Equity,4.401959,-20.0,...,1.0,0.000000,-4.401959,-100.000000,5.0,-50.0,0.0,9.673239e+00,5.271280,119.748513
4,2025-09-05,1.054599,1.114748,0.479380,0.030569,0.095607,call,FX,0.011694,-20.0,...,1.0,0.485526,0.473832,4051.937923,5.0,-50.0,0.0,8.797493e-03,-0.002896,-24.768878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2025-09-16,55.265764,59.074121,0.706291,0.043157,2.500000,put,Commodity,40.786564,-20.0,...,1.0,55.922971,15.136407,37.111257,5.0,-50.0,0.0,3.419638e+01,-6.590185,-16.157735
196,2025-09-17,100.961208,93.299603,0.246211,0.037852,0.207502,put,Equity,1.079244,-20.0,...,1.0,58.021275,56.942031,5276.103306,5.0,-50.0,0.0,0.000000e+00,-1.079244,-100.000000
197,2025-09-18,95.377247,99.664774,0.224177,0.022612,0.234969,put,Equity,6.467507,-20.0,...,1.0,63.390103,56.922596,880.131947,5.0,-50.0,0.0,8.836698e-100,-6.467507,-100.000000
198,2025-09-19,1.172349,1.201867,1.783358,0.034635,0.081875,put,FX,0.031779,-20.0,...,1.0,1.072182,1.040403,3273.891189,5.0,-50.0,0.0,0.000000e+00,-0.031779,-100.000000


In [26]:
final_df['worst_case_col'] = final_df[pnl_cols].idxmin(axis=1)
final_df['worst_case_pnl'] = final_df[pnl_cols].min(axis=1)
def get_worst_scenario_name(col_val):
    if pd.isna(col_val):
        return "None"
    return str(col_val).replace("_PnL", "")

final_df['worst_case_scenario'] = final_df['worst_case_col'].apply(get_worst_scenario_name)
def get_worst_pnl_pct(row):
    scen = row['worst_case_scenario']
    if scen == "None":
        return np.nan
    col_name = f"{scen}_PnL%"
    return row.get(col_name, np.nan)

final_df['worst_case_pnl_pct'] = final_df.apply(get_worst_pnl_pct, axis=1)


/var/folders/r_/js4cr3ps49j4qkf0dy2w4lm00000gn/T/ipykernel_22509/1042820306.py:1: FutureWarning: The behavior of DataFrame.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  final_df['worst_case_col'] = final_df[pnl_cols].idxmin(axis=1)


In [ ]:
final_df = final_df.drop(columns=['worst_case_col'])

['Dot-com Crash (2000)',
 'Volatility Collapse',
 'Rate Shock (+200bps)',
 'Black Monday (1987)',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'None',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Black Monday (1987)',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Rate Shock (+200bps)',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'None',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collapse',
 'Volatility Collap

## _get_scenario_pnl

In [7]:
from bsm_multi_agents.mcp.pricing_validator import (
    _bsm_price,_bsm_delta,_bsm_gamma,_bsm_vega,_bsm_theta
)

In [13]:
input_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

df = pd.read_csv(input_path)
row = df.iloc[0]
option_data = row
scenario = {
    "name": "Black Monday (1987)", 
    "spot_change": -0.20, 
    "vol_change": 0.50, "rate_change": -0.005,
    "days_passed": 1.0,
    "rate_change": 0,
}

In [14]:
opt_type = option_data['option_type']
S0 = float(option_data['S'])
K = float(option_data['K'])
T0 = float(option_data['T'])
r = float(option_data['r'])
sigma0 = float(option_data['sigma'])

# Base case
V0 = _bsm_price(opt_type, S0, K, T0, r, sigma0)
delta0 = _bsm_delta(opt_type, S0, K, T0, r, sigma0)
gamma0 = _bsm_gamma(S0, K, T0, r, sigma0)
vega0 = _bsm_vega(S0, K, T0, r, sigma0)
theta0 = _bsm_theta(opt_type, S0, K, T0, r, sigma0)

spot_change = scenario.get('spot_change', 0.0)
vol_change = scenario.get('vol_change', 0.0)
days_passed = scenario.get('days_passed', 0.0)
rate_change = scenario.get('rate_change', 0.0)

S1 = S0*(1+spot_change)
sigma1 = sigma0*(1+vol_change)
T1 = max(0, T0 - days_passed / 365.0)
r1 = r*(1+rate_change)

# New option price
V1 = _bsm_price(opt_type, S1, K, T1, r1, sigma1)
actual_pnl = V1 - V0

# Greeks-based P&L estimate
spot_move = S1 - S0
vol_move = sigma1 - sigma0
rate_move = r1 - r
time_decay = days_passed / 365.0

# P&L components
delta_pnl = delta0 * spot_move
gamma_pnl = 0.5 * gamma0 * (spot_move ** 2)
vega_pnl = vega0 * vol_move
theta_pnl = theta0 * time_decay

# Total estimated P&L
estimated_pnl = delta_pnl + gamma_pnl + vega_pnl + theta_pnl
pnl_error = actual_pnl - estimated_pnl

# Realized variance (gamma P&L proxy)
realized_var = (spot_move / S0) ** 2 if S0 != 0 else 0

# Delta-hedged P&L (excluding delta component)
hedged_pnl = gamma_pnl + vega_pnl + theta_pnl

{
                "spot_level": S1,
                "spot_move": spot_move,
                "vol_level": sigma1,
                "vol_move": vol_move,
                "days_passed": days_passed,
                "rate_level": r1,
                "rate_move": rate_move,
                "base_price": V0,
                "new_price": V1,
                "actual_pnl": actual_pnl,
                "delta_pnl": delta_pnl,
                "gamma_pnl": gamma_pnl,
                "vega_pnl": vega_pnl,
                "theta_pnl": theta_pnl,
                "estimated_pnl": estimated_pnl,
                "pnl_error": pnl_error,
                "realized_variance": realized_var,
                "delta_hedged_pnl": hedged_pnl
            }

{'spot_level': 0.8998685661204493,
 'spot_move': -0.2249671415301122,
 'vol_level': 3.75,
 'vol_move': 1.25,
 'days_passed': 1.0,
 'rate_level': 0.0162397808134481,
 'rate_move': 0.0,
 'base_price': array(0.63655954),
 'new_price': array(0.65812347),
 'actual_pnl': np.float64(0.021563930083646987),
 'delta_pnl': np.float64(-0.17590774427623396),
 'gamma_pnl': np.float64(0.004210418369520421),
 'vega_pnl': np.float64(0.002608060877178551),
 'theta_pnl': np.float64(-4.967714760069226e-06),
 'estimated_pnl': np.float64(-0.16909423274429508),
 'pnl_error': np.float64(0.19065816282794207),
 'realized_variance': 0.03999999999999997,
 'delta_hedged_pnl': np.float64(0.006813511531938902)}

In [6]:
S1

1.1360840647270671

## _run_pnl_test

In [26]:
from bsm_multi_agents.mcp import pricing_validator
importlib.reload(pricing_validator)
from bsm_multi_agents.mcp.pricing_validator import (
    _bsm_price,_bsm_delta,_bsm_gamma,_bsm_vega,_bsm_theta,_get_option_pnl_under_scenario
)

In [27]:
input_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [28]:
df = pd.read_csv(input_path)
row = df.iloc[0]
option_data = row
scenarios = [
    {"name": "Black Monday (1987)", "spot_change": -0.20, "vol_change": 0.50, "rate_change": -0.005},
    {"name": "Dot-com Crash (2000)", "spot_change": -0.70, "vol_change": 2.00, "rate_change": -0.02},
    {"name": "2008 Financial Crisis", "spot_change": -0.50, "vol_change": 1.50, "rate_change": -0.01},
    {"name": "VIX Spike (No Stock Move)", "spot_change": 0.0, "vol_change": 1.00, "rate_change": 0.0},
    {"name": "Rate Shock (+200bps)", "spot_change": 0.0, "vol_change": 0.0, "rate_change": 0.02},
    {"name": "Liquidation Scenario", "spot_change": -0.30, "vol_change": 3.00, "rate_change": 0.01},
    {"name": "Volatility Collapse", "spot_change": 0.05, "vol_change": -0.50, "rate_change": 0.0},
]

In [29]:
required_fields = ['option_type', 'S', 'K', 'T', 'r', 'sigma']
not all(field in option_data for field in required_fields)

False

In [30]:
details = [_get_option_pnl_under_scenario(row, s) for s in scenarios]

In [31]:
num_scenarios = len(details)
avg_actual_pnl = np.mean([d['actual_pnl'] for d in details])
max_actual_pnl = np.max([d['actual_pnl'] for d in details])
min_actual_pnl = np.min([d['actual_pnl'] for d in details])
avg_pnl_error = np.mean([d['pnl_error'] for d in details])
max_pnl_error = np.max(np.abs([d['pnl_error'] for d in details]))
avg_delta_hedged_pnl = np.mean([d['delta_hedged_pnl'] for d in details])

In [32]:
pd.Series({
    'num_scenarios': num_scenarios,
    'avg_actual_pnl': avg_actual_pnl,
    'max_actual_pnl': max_actual_pnl,
    'min_actual_pnl': min_actual_pnl,
    'avg_pnl_error': avg_pnl_error,
    'max_pnl_error': max_pnl_error,
    'avg_delta_hedged_pnl': avg_delta_hedged_pnl,
    'details': details
})

num_scenarios                                                           7
avg_actual_pnl                                                  -0.021918
max_actual_pnl                                                   0.357539
min_actual_pnl                                                  -0.310115
avg_pnl_error                                                    0.166693
max_pnl_error                                                    0.388011
avg_delta_hedged_pnl                                             0.018709
details                 [{'scenario_name': 'Black Monday (1987)', 'spo...
dtype: object

## run_pnl_test

In [33]:
from bsm_multi_agents.mcp import pricing_validator
importlib.reload(pricing_validator)
from bsm_multi_agents.mcp.pricing_validator import (
    _get_option_pnl
)

In [34]:
input_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [38]:
df = pd.read_csv(input_path)
scenarios = DEFAULT_SCENARIOS
required_cols = ['option_type', 'S', 'K', 'T', 'r', 'sigma']
missing = [c for c in required_cols if c not in df.columns]

In [41]:
results = df.apply(lambda r: _get_option_pnl(r, scenarios), axis=1)
results.head()

/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/pricing_validator.py:36: RuntimeWarning: invalid value encountered in log
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrtT)
/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/pricing_validator.py:66: RuntimeWarning: invalid value encountered in log
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrtT)
/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/pricing_validator.py:90: RuntimeWarning: invalid value encountered in log
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrtT)
/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/pricing_validator.py:104: RuntimeWarning: invalid value encountered in log
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrtT)
/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/pricing_validator.py:119: Ru

,num_scenarios,avg_actual_pnl,max_actual_pnl,min_actual_pnl,avg_pnl_error,max_pnl_error,avg_delta_hedged_pnl,details
0,7,-0.021918,0.357539,-0.310115,0.166693,0.388011,0.018709,"[{'scenario_name': 'Black Monday (1987)', 'spo..."
1,7,-0.535333,7.197798,-6.064298,-2.670909,25.171305,13.441498,"[{'scenario_name': 'Black Monday (1987)', 'spo..."
2,7,-2.836823,5.413895,-10.545406,1.375481,12.080192,12.221446,"[{'scenario_name': 'Black Monday (1987)', 'spo..."
3,7,-1.762365,5.271280,-4.401959,23.087955,69.395962,0.000000,"[{'scenario_name': 'Black Monday (1987)', 'spo..."
4,7,-0.000875,0.025833,-0.011694,-0.265679,1.119058,0.334356,"[{'scenario_name': 'Black Monday (1987)', 'spo..."


In [42]:
pd.concat([df, results], axis=1)

,date,S,K,T,r,sigma,option_type,asset_class,num_scenarios,avg_actual_pnl,max_actual_pnl,min_actual_pnl,avg_pnl_error,max_pnl_error,avg_delta_hedged_pnl,details
0,2025-09-01,1.124836,1.147031,0.396435,0.016240,2.500000,call,FX,7,-0.021918,0.357539,-0.310115,0.166693,0.388011,0.018709,"[{'scenario_name': 'Black Monday (1987)', 'spo..."
1,2025-09-02,98.617357,105.839622,1.242119,0.038323,0.164521,call,Equity,7,-0.535333,7.197798,-6.064298,-2.670909,25.171305,13.441498,"[{'scenario_name': 'Black Monday (1987)', 'spo..."
2,2025-09-03,95.305256,89.270613,0.678060,0.030990,0.195456,call,Equity,7,-2.836823,5.413895,-10.545406,1.375481,12.080192,12.221446,"[{'scenario_name': 'Black Monday (1987)', 'spo..."
3,2025-09-04,105.425600,101.023642,-0.100000,0.015580,0.257986,call,Equity,7,-1.762365,5.271280,-4.401959,23.087955,69.395962,0.000000,"[{'scenario_name': 'Black Monday (1987)', 'spo..."
4,2025-09-05,1.054599,1.114748,0.479380,0.030569,0.095607,call,FX,7,-0.000875,0.025833,-0.011694,-0.265679,1.119058,0.334356,"[{'scenario_name': 'Black Monday (1987)', 'spo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2025-09-16,55.265764,59.074121,0.706291,0.043157,2.500000,put,Commodity,7,8.051269,16.499773,-18.048973,5.413239,17.504346,0.674471,"[{'scenario_name': 'Black Monday (1987)', 'spo..."
196,2025-09-17,100.961208,93.299603,0.246211,0.037852,0.207502,put,Equity,7,20.287437,61.084363,-1.073128,-0.204766,15.254289,16.165540,"[{'scenario_name': 'Black Monday (1987)', 'spo..."
197,2025-09-18,95.377247,99.664774,0.224177,0.022612,0.234969,put,Equity,7,22.130147,64.090635,-4.714140,-12.167195,57.342096,20.456554,"[{'scenario_name': 'Black Monday (1987)', 'spo..."
198,2025-09-19,1.172349,1.201867,1.783358,0.034635,0.081875,put,FX,7,0.254772,0.747801,-0.030162,-0.088610,0.508739,0.247361,"[{'scenario_name': 'Black Monday (1987)', 'spo..."


## write_report_to_word

# Agent

## pricing_calculator_agent_node

### Inside Node

In [4]:
from bsm_multi_agents.config.llm_config import get_llm
from bsm_multi_agents.agents.utils import extract_mcp_content, load_tools_from_mcp_and_local

In [5]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
local_tool_folder_path = os.path.join(project_path, "src/bsm_multi_agents/tools")
final_report_path = os.path.join(project_path, "data/output/final_report.docx")

In [6]:
initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_folder_path=local_tool_folder_path,
    final_report_path=final_report_path,
)
state = initial_state

In [7]:
errors = state.get("errors", [])
"csv_file_path" not in state or not state["csv_file_path"]
server_path = state.get("server_path")
output_dir = state.get("output_dir")

In [8]:
local_tool_folder_path = state.get("local_tool_folder_path", "")
langchain_tools = load_tools_from_mcp_and_local(server_path, local_tool_folder_path)
langchain_tools

[StructuredTool(name='calculate_greeks_to_file', description='Reads the CSV at input_path, calculates greeks, and saves the result to output_dir.\n    Returns the path to the result file.', args_schema=<class 'bsm_multi_agents.agents.mcp_adapter.calculate_greeks_to_fileInput'>, func=<function mcp_tool_to_langchain_tool.<locals>.tool_func at 0x10c6bd580>),
 StructuredTool(name='validate_greeks_to_file', description='Validate Greeks for ALL options from CSV data.\n\n    For each option:\n    - Validates: BSM_price > 0\n    - Validates: delta in [0,1] for calls, [-1,0] for puts\n    - Validates: gamma >= 0, vega >= 0\n\n    Args:\n        state: InjectedState, state from the workflow, which contains csv_data\n\n\n    Returns:\n        JSON string containing validate_results', args_schema=<class 'bsm_multi_agents.agents.mcp_adapter.validate_greeks_to_fileInput'>, func=<function mcp_tool_to_langchain_tool.<locals>.tool_func at 0x10ccdede0>),
 StructuredTool(name='run_stress_test_to_file', d

In [8]:
llm = get_llm().bind_tools(langchain_tools)

In [ ]:
system_prompt = (
    "You are a quantitative calculator agent. "
    "You have access to tools specifically for Greeks calculation via an MCP server, as well as local math tools. "
    "You operate in a ReAct loop: you can call a tool, see the result, and then decide to call another tool or finish. "
    "Use the available tools to process these requests sequentially or in parallel if appropriate. "
    "If you have multiple distinct tasks (e.g. Calculate A, then Calculate B), handle them one by one or together.\n"
    "IMPORTANT: When you have completed ALL requested tasks and saved the results, you MUST output a final text response (e.g. 'Calibration and calculation complete.') with NO tool calls. This will signal the workflow to proceed."
)

user_prompt = (
    f"Input CSV File: {state['csv_file_path']}\n"
    f"Output Directory: {output_dir}\n\n"
    "Please calculate the Greeks for the options in the input CSV file. "
    "Save the results to the output directory. "
    "Ensure you call the calculation tools."
)

# user_prompt = (
#     "Give me answer of 2 plus 4."
# )

messages = list(state.get("messages", []))
if not messages:
    messages.append(SystemMessage(content=system_prompt))
    messages.append(HumanMessage(content=user_prompt))

In [10]:
ai_msg = llm.invoke(messages)
messages.append(ai_msg)
state["messages"] = messages

In [11]:
print(f"\n>>> [Pricing Calculator Agent] Decide to use tools: {[tool['name'] for tool in ai_msg.tool_calls]}")


>>> [Pricing Calculator Agent] Decide to use tools: ['calculate_greeks_to_file']


### Node Level

In [12]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
local_tool_folder_path = os.path.join(project_path, "src/bsm_multi_agents/tools")
final_report_path = os.path.join(project_path, "data/output/final_report.docx")

In [13]:
from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import pricing_calculator_agent_node

In [14]:
initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_folder_path=local_tool_folder_path,
    final_report_path=final_report_path,
)

In [15]:
state = pricing_calculator_agent_node(initial_state)


>>> [Pricing Calculator Agent] Starting planning...
>>> [Pricing Calculator Agent] Decide to use tools: ['calculate_greeks_to_file']


In [16]:
state['messages'][-1].tool_calls

[{'name': 'calculate_greeks_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache'},
  'id': '251654fd-e4a3-410f-a0f2-b50b1753b518',
  'type': 'tool_call'}]

## pricing_calculator_tool_node

### Inside Node

In [45]:
from bsm_multi_agents.config.llm_config import get_llm
from bsm_multi_agents.agents.mcp_adapter import list_mcp_tools_sync

In [46]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
local_tool_folder_path = os.path.join(project_path, "src/bsm_multi_agents/tools")
final_report_path = os.path.join(project_path, "data/output/final_report.docx")



from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import pricing_calculator_agent_node

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_folder_path=local_tool_folder_path,
    final_report_path=final_report_path,
)

state = pricing_calculator_agent_node(initial_state)


>>> [Pricing Calculator Agent] Starting planning...


  + Exception Group Traceback (most recent call last):
  |   File "/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/r_/js4cr3ps49j4qkf0dy2w4lm00000gn/T/ipykernel_12491/1447707534.py", line 21, in <module>
  |     state = pricing_calculator_agent_node(initial_state)
  |             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/agents/pricing_calculator_node.py", line 38, in pricing_calculator_agent_node
  |     langchain_tools = load_tools_from_mcp_and_local(server_path, local_tool_folder_path)
  |                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/agents/utils.py", line 134, in load_tools_from_mcp_and

In [19]:
from bsm_multi_agents.agents.mcp_adapter import call_mcp_tool
from bsm_multi_agents.agents.utils import extract_mcp_content,call_local_tool,load_local_tools_from_file

In [20]:
errors = state.get("errors", [])
messages = list(state.get("messages", []))

In [21]:
last_msg = messages[-1]
not hasattr(last_msg, "tool_calls") or not last_msg.tool_calls

False

In [22]:
server_path = state.get("server_path")

In [23]:
"tool_outputs" not in state or state["tool_outputs"] is None

True

In [24]:
tool_call = last_msg.tool_calls[0]
tool_outputs_msgs = []
tool_name = tool_call["name"]
args = tool_call["args"]
call_id = tool_call["id"]
print(f">>> [Pricing Calculator Tool] Executing tool calls: {tool_name}")


>>> [Pricing Calculator Tool] Executing tool calls: calculate_greeks_to_file


In [25]:
# local_tool_paths = state.get("local_tool_paths", [])
# raw_result = call_local_tool(tool_name, args=args, local_tool_paths=local_tool_paths)
# raw_result

In [26]:
raw_result = call_mcp_tool(tool_name, server_path, args)
result_text = extract_mcp_content(raw_result)
result_text

'/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results.csv'

In [27]:
tool_outputs_msgs.append(ToolMessage(content=result_text, tool_call_id=call_id, name=tool_name))
state["greeks_results_path"] = result_text.strip()

### Node Level

In [30]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
local_tool_folder_path = os.path.join(project_path, "src/bsm_multi_agents/tools")
final_report_path = os.path.join(project_path, "data/output/final_report.docx")

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_folder_path=local_tool_folder_path,
    final_report_path=final_report_path,
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(state)


>>> [Pricing Calculator Agent] Starting planning...
>>> [Pricing Calculator Agent] Decide to use tools: ['calculate_greeks_to_file']

>>> [Pricing Calculator Tool] Executing tool calls...
>>> [Pricing Calculator Tool] Executing tool calls: calculate_greeks_to_file


In [31]:
state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/server.py',
 'local_tool_folder_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/tools',
 'final_report_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/output/final_report.docx',
 'messages': [SystemMessage(content="You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server, as well as local math tools. You operate in a ReAct loop: you can call a tool, see the result, and then decide to call another tool or finish. Use the available tools to process these requests sequentially or in parallel if appropriate. If you have multiple distinct tasks (e.g. Calculate A, then Calculate B), handle them one 

## pricing_validator_agent_node

### Inside Node

In [9]:
from bsm_multi_agents.config.llm_config import get_llm
from bsm_multi_agents.agents.mcp_adapter import call_mcp_tool
from bsm_multi_agents.agents.utils import load_tools_from_mcp_and_local

In [10]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
local_tool_folder_path = os.path.join(project_path, "src/bsm_multi_agents/tools")
final_report_path = os.path.join(project_path, "data/output/final_report.docx")



from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_folder_path=local_tool_folder_path,
    final_report_path=final_report_path
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(initial_state)


>>> [Pricing Calculator Agent] Starting planning...
>>> [Pricing Calculator Agent] Decide to use tools: ['calculate_greeks_to_file']

>>> [Pricing Calculator Tool] Executing tool calls...
>>> [Pricing Calculator Tool] Executing tool calls: calculate_greeks_to_file


In [11]:
errors = state.get("errors", [])
"greeks_results_path" not in state or not state["greeks_results_path"]

False

In [12]:
server_path = state.get("server_path")
output_dir = state.get("output_dir")

In [13]:
local_tool_folder_path = state.get("local_tool_folder_path", "")
langchain_tools = load_tools_from_mcp_and_local(server_path, local_tool_folder_path) 
langchain_tools

[StructuredTool(name='calculate_greeks_to_file', description='Reads the CSV at input_path, calculates greeks, and saves the result to output_dir.\n    Returns the path to the result file.', args_schema=<class 'bsm_multi_agents.agents.mcp_adapter.calculate_greeks_to_fileInput'>, func=<function mcp_tool_to_langchain_tool.<locals>.tool_func at 0x105b94680>),
 StructuredTool(name='validate_greeks_to_file', description='Validate Greeks for ALL options from CSV data.\n\n    For each option:\n    - Validates: BSM_price > 0\n    - Validates: delta in [0,1] for calls, [-1,0] for puts\n    - Validates: gamma >= 0, vega >= 0\n\n    Args:\n        state: InjectedState, state from the workflow, which contains csv_data\n\n\n    Returns:\n        JSON string containing validate_results', args_schema=<class 'bsm_multi_agents.agents.mcp_adapter.validate_greeks_to_fileInput'>, func=<function mcp_tool_to_langchain_tool.<locals>.tool_func at 0x11cbc23e0>),
 StructuredTool(name='run_stress_test_to_file', d

In [14]:
llm = get_llm().bind_tools(langchain_tools)

In [15]:
system_prompt = (
    "You are a quantitative validator agent. "
    "You have access to tools specifically for Greeks calculation via an MCP server, as well as local math tools. "
    "You operate in a ReAct loop: you can call a tool, see the result, and then decide to call another tool or finish. "
    "Use the available tools to process the requested data. "
    "Your validation process MUST include:\n"
    "1. **Greek Validation**: Validate ranges for Delta, Gamma, Vega, etc.\n"
    "2. **Stress Testing**: Run stress tests with market scenarios.\n"
    "3. **P&L Attribution**: Execute P&L analysis and attribution tests.\n"
    "If you have multiple distinct tasks, handle them one by one or together.\n"
    "IMPORTANT: When you have completed ALL requested tasks and saved the results, you MUST output a final text response (e.g. 'Calibration and calculation complete.') with NO tool calls. This will signal the workflow to proceed."
)

user_prompt = (
    f"Input CSV File: {state['greeks_results_path']}\n"
    f"Output Directory: {output_dir}\n\n"
    "Please validate the Greeks for the options in the input CSV file. "
    "Additionally, run stress tests and execute P&L attribution tests using the appropriate tools. "
    "Save all results to the output directory. "
)
messages = list(state.get("messages", []))
messages.append(SystemMessage(content=system_prompt))
messages.append(HumanMessage(content=user_prompt))


In [16]:
ai_msg = llm.invoke(messages)

In [17]:
ai_msg.tool_calls

[{'name': 'validate_greeks_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache'},
  'id': 'b4b9f2ac-e8d5-4e9a-9595-3bd1aafc7954',
  'type': 'tool_call'},
 {'name': 'run_stress_test_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache',
   'scenarios': 'default'},
  'id': '9965b4a2-4194-44fc-be87-666d964f238d',
  'type': 'tool_call'},
 {'name': 'run_pnl_attribution_test_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache',
   'scenarios': 'default'},
  'id': '34052b02-8c36-4940-9451-e3d37f20718e'

### Node Level

In [18]:
final_report_path

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)


from bsm_multi_agents.agents.pricing_validator_node import (
    pricing_validator_agent_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_folder_path=local_tool_folder_path,
    final_report_path=final_report_path,
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(state)
state = pricing_validator_agent_node(state)
state


>>> [Pricing Calculator Agent] Starting planning...
>>> [Pricing Calculator Agent] Decide to use tools: ['calculate_greeks_to_file']

>>> [Pricing Calculator Tool] Executing tool calls...
>>> [Pricing Calculator Tool] Executing tool calls: calculate_greeks_to_file

>>> [Pricing Validator Agent] Starting validation planning...
>>> [Pricing Validator Agent] Decide to use tools: ['validate_greeks_to_file', 'run_stress_test_to_file', 'run_pnl_attribution_test_to_file']


{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/server.py',
 'local_tool_folder_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/tools',
 'final_report_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/output/final_report.docx',
 'messages': [SystemMessage(content="You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server, as well as local math tools. You operate in a ReAct loop: you can call a tool, see the result, and then decide to call another tool or finish. Use the available tools to process these requests sequentially or in parallel if appropriate. If you have multiple distinct tasks (e.g. Calculate A, then Calculate B), handle them one 

In [19]:
state['messages'][-1].tool_calls

[{'name': 'validate_greeks_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache'},
  'id': '8565973b-da91-4a06-bf95-2bee8cbba615',
  'type': 'tool_call'},
 {'name': 'run_stress_test_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache',
   'scenarios': 'default'},
  'id': '610c0e0b-6a2a-4c6a-b6ce-891f79b156be',
  'type': 'tool_call'},
 {'name': 'run_pnl_attribution_test_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache',
   'scenarios': 'default'},
  'id': '83646d82-303a-401b-87ef-9c1d688140f9'

## pricing_validator_tool_node

### Inside Node

In [4]:
from bsm_multi_agents.agents.mcp_adapter import call_mcp_tool
from bsm_multi_agents.agents.utils import extract_mcp_content, call_local_tool

In [5]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
local_tool_folder_path = os.path.join(project_path, "src/bsm_multi_agents/tools")
final_report_path = os.path.join(project_path, "data/output/final_report.docx")



from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)


from bsm_multi_agents.agents.pricing_validator_node import (
    pricing_validator_agent_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_folder_path=local_tool_folder_path,
    final_report_path=final_report_path
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(state)
state = pricing_validator_agent_node(state)
state


>>> [Pricing Calculator Agent] Starting planning...
>>> [Pricing Calculator Agent] Decide to use tools: ['calculate_greeks_to_file']

>>> [Pricing Calculator Tool] Executing tool calls...
>>> [Pricing Calculator Tool] Executing tool calls: calculate_greeks_to_file

>>> [Pricing Validator Agent] Starting validation planning...
>>> [Pricing Validator Agent] Decide to use tools: ['validate_greeks_to_file', 'run_stress_test_to_file', 'run_pnl_attribution_test_to_file']


{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/server.py',
 'local_tool_folder_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/tools',
 'final_report_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/output/final_report.docx',
 'messages': [SystemMessage(content="You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server, as well as local math tools. You operate in a ReAct loop: you can call a tool, see the result, and then decide to call another tool or finish. Use the available tools to process these requests sequentially or in parallel if appropriate. If you have multiple distinct tasks (e.g. Calculate A, then Calculate B), handle them one 

In [6]:
errors = state.get("errors", [])
messages = list(state.get("messages", []))
last_msg = messages[-1]
server_path = state.get("server_path")
tool_outputs_msgs = []

In [7]:
last_msg.tool_calls

[{'name': 'validate_greeks_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache'},
  'id': 'b55bebaa-bfa4-4e10-9f8a-52c27098f4f9',
  'type': 'tool_call'},
 {'name': 'run_stress_test_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache'},
  'id': '3cd0ee78-fc4f-4e8c-b59e-c6f9733ad77f',
  'type': 'tool_call'},
 {'name': 'run_pnl_attribution_test_to_file',
  'args': {'input_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results.csv',
   'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache'},
  'id': '25ab0de6-23bd-4135-a4b3-bd61dc402ada',
  'type': 'tool_call'}]

In [11]:
tool_call = last_msg.tool_calls[2]
tool_name = tool_call["name"]
args = tool_call["args"]
call_id = tool_call["id"]
print(f">>> [Pricing Validator Tool] Executing tool calls: {tool_name}")


>>> [Pricing Validator Tool] Executing tool calls: run_pnl_attribution_test_to_file


In [12]:
# local_tool_paths = state.get("local_tool_paths", [])
# raw_result = call_local_tool(tool_name, args=args, local_tool_paths=local_tool_paths)
# result_text = str(raw_result)

In [13]:
raw_result = call_mcp_tool(tool_name, server_path, args)
result_text = extract_mcp_content(raw_result)
result_text

'/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results_pnl_test_results.csv'

In [13]:
for tool_call in last_msg.tool_calls:
    tool_name = tool_call["name"]
    args = tool_call["args"]
    call_id = tool_call["id"]
    print(f">>> [Pricing Validator Tool] Executing tool calls: {tool_name}")
    
    try:
        # 1. Try Local Tool First
        local_tool_paths = state.get("local_tool_paths", [])
        try:
            raw_result = call_local_tool(tool_name, args=args, local_tool_paths=local_tool_paths)
            result_text = str(raw_result)
        except LookupError:
            # 2. Fallback to MCP Tool
            # print(f"Tool {tool_name} not found locally, trying MCP...")
            raw_result = call_mcp_tool(tool_name, server_path, args)
            result_text = extract_mcp_content(raw_result)
            print(result_text)
        
        # Create ToolMessage
        tool_outputs_msgs.append(ToolMessage(content=result_text, tool_call_id=call_id, name=tool_name))
        
        # Generic Output Handling
        state["validate_results_path"] = result_text.strip()

    except Exception as e:
        err_msg = f"Error executing {tool_name}: {e}"
        errors.append(err_msg)
        tool_outputs_msgs.append(ToolMessage(content=err_msg, tool_call_id=call_id, is_error=True))
            

>>> [Pricing Validator Tool] Executing tool calls: validate_greeks_to_file
/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results_validate_results.csv
>>> [Pricing Validator Tool] Executing tool calls: run_stress_test_to_file
Error: Stress test execution failed: 'str' object has no attribute 'get'
>>> [Pricing Validator Tool] Executing tool calls: run_pnl_attribution_test_to_file
Error executing tool run_pnl_attribution_test_to_file: 1 validation error for run_pnl_attribution_test_to_fileArguments
scenarios
  Input should be a valid list [type=list_type, input_value='default', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/list_type


In [51]:
tool_outputs_msgs

[ToolMessage(content='/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache/dummy_options_greeks_results_validate_results.csv', name='validate_greeks_to_file', tool_call_id='a0c06307-86e5-44d3-bcf5-8e4bce01ae22')]

In [52]:
state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/server.py',
 'local_tool_folder_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/tools',
 'final_report_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/output/final_report.docx',
 'messages': [SystemMessage(content="You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server, as well as local math tools. You operate in a ReAct loop: you can call a tool, see the result, and then decide to call another tool or finish. Use the available tools to process these requests sequentially or in parallel if appropriate. If you have multiple distinct tasks (e.g. Calculate A, then Calculate B), handle them one 

### Node Level

In [53]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
local_tool_folder_path = os.path.join(project_path, "src/bsm_multi_agents/tools")
final_report_path = os.path.join(project_path, "data/output/final_report.docx")

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)


from bsm_multi_agents.agents.pricing_validator_node import (
    pricing_validator_agent_node,
    pricing_validator_tool_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_folder_path=local_tool_folder_path,
    final_report_path=final_report_path
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(state)
state = pricing_validator_agent_node(state)
state = pricing_validator_tool_node(state)
state


>>> [Pricing Calculator Agent] Starting planning...
>>> [Pricing Calculator Agent] Decide to use tools: ['calculate_greeks_to_file']

>>> [Pricing Calculator Tool] Executing tool calls...
>>> [Pricing Calculator Tool] Executing tool calls: calculate_greeks_to_file

>>> [Pricing Validator Agent] Starting validation planning...
>>> [Pricing Validator Agent] Decide to use tools: ['validate_greeks_to_file']

>>> [Pricing Validator Tool] Executing validation calls...
>>> [Pricing Validator Tool] Executing tool calls: validate_greeks_to_file


{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/server.py',
 'local_tool_folder_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/tools',
 'final_report_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/output/final_report.docx',
 'messages': [SystemMessage(content="You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server, as well as local math tools. You operate in a ReAct loop: you can call a tool, see the result, and then decide to call another tool or finish. Use the available tools to process these requests sequentially or in parallel if appropriate. If you have multiple distinct tasks (e.g. Calculate A, then Calculate B), handle them one 

## report_generator_node

### Inside Node

In [121]:
from bsm_multi_agents.config.llm_config import get_llm

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
local_tool_folder_path = os.path.join(project_path, "src/bsm_multi_agents/tools")
final_report_path = os.path.join(project_path, "data/output/final_report.docx")

from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)


from bsm_multi_agents.agents.pricing_validator_node import (
    pricing_validator_agent_node,
    pricing_validator_tool_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_folder_path=local_tool_folder_path,
    final_report_path=final_report_path,
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(state)
state = pricing_validator_agent_node(state)
state = pricing_validator_tool_node(state)
state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/server.py',
 'local_tool_paths': ['/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/tools/my_add.py'],
 'final_report_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/output/final_report.docx',
 'messages': [SystemMessage(content='You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server. Use the available tools to process the requested data. If you are confident, you can run all tools in parallel.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Input CSV File: /Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv\nOutput Directory: /Users/yifanli/Githu

In [123]:
errors = state.get("errors", [])
csv_path = state.get("validate_results_path")
output_dir = state.get("output_dir")
validate_results_path = state.get("validate_results_path")
final_report_path = state.get("final_report_path")


In [124]:
title: str = "Ongoing Monitoring Analysis Report"
model_name: str = "Option Pricing, BSM"
author_name: str = "John Doe"
group_name: str = "Front Desk Modeling and Analytics"
version: str = "v1.0"
section1_heading: str = "1. Introduction"
section1_paragraph: Optional[str] = None
section2_heading: str = "2. Summary of Analysis"

In [125]:
llm = get_llm()

In [126]:
doc = Document()
# Title
title_run = doc.add_paragraph().add_run(title)
title_run.bold = True
title_run.font.size = doc.styles["Title"].font.size
doc.paragraphs[-1].alignment = 1  # 0=left, 1=center

# Subtitle (Model Name)
subtitle_para = doc.add_paragraph()
subtitle_run = subtitle_para.add_run(model_name)
subtitle_para.alignment = 1
subtitle_run.bold = True

doc.add_paragraph("")  # spacer

# Author
author_para = doc.add_paragraph()
author_para.add_run("Author: ").bold = True
author_para.add_run(author_name).italic = False
author_para.alignment = 1

# Group
group_para = doc.add_paragraph()
group_para.add_run("Group: ").bold = True
group_para.add_run(group_name)
group_para.alignment = 1

# Report date
date_str = datetime.date.today().strftime("%B %d, %Y")
date_para = doc.add_paragraph()
date_para.add_run("Report Date: ").bold = True
date_para.add_run(date_str)
date_para.alignment = 1

# Version
version_para = doc.add_paragraph()
version_para.add_run("Document Version: ").bold = True
version_para.add_run(version)
version_para.alignment = 1

# Page break after title page
doc.add_page_break()

# ============================
# TABLE OF CONTENTS
# ============================
doc.add_heading("Table of Contents", level=1)

toc_para = doc.add_paragraph()
fld = OxmlElement("w:fldSimple")
fld.set(qn("w:instr"), 'TOC \\o "1-3" \\h \\z \\u')
toc_para._p.append(fld)

doc.add_page_break()

# ============================
# SECTION 1
# ============================
if not section1_paragraph:
    section1_paragraph = (
        "This section provides contextual background, objectives, and relevant "
        "considerations for the ongoing monitoring analysis. Subsequent sections "
        "expand on methodology, insights, and results."
    )

doc.add_heading(section1_heading, level=1)
doc.add_paragraph(section1_paragraph)

# ============================
# SECTION 2 – Refined Summary
# ============================
doc.add_heading(section2_heading, level=1)



# Read CSV
# with open(pricing_path, newline="", encoding="utf-8") as f:
#     reader = csv.reader(f)
#     rows = list(reader)
df_validate = pd.read_csv(validate_results_path)

for asset in ["FX", "Equity", "Commodity"]:
    doc.add_paragraph("The pricing output of "+asset+ " listed in the below table,")
    df = df_validate[df_validate["asset_class"] == asset]
    df = df.sort_values("T")
    # df = df.dropna()
    
    fig, ax = plt.subplots()
    df_call = df[df["option_type"] == "call"]
    ax.plot(df_call["T"], df_call["BSM_price"], label = "call")
    df_put = df[df["option_type"] == "put"]
    ax.plot(df_put["T"], df_put["BSM_price"], label = "put")

    ax.set_xlabel("Time to Maturity (T)")
    ax.set_ylabel("Option Price (BSM)")
    ax.set_title(f"Option Pricing Curve – {asset}")  
    ax.legend()

    # Save figure to memory (PNG bytes)
    img_stream = BytesIO()
    fig.savefig(img_stream, format="png", dpi=200, bbox_inches="tight")
    plt.close(fig)
    img_stream.seek(0)

    system_prompt = (
        "Please summmarize the option pricing results (pull and call) from the tables with the following topics"
        "1. Overall Data Quality"
        "2. Pricing Level by Asset Class"
        "3. Term Structure (Price vs Maturity)"
        "4. Call vs Put Behavior"
        "5. Model Consistency"
        "6. Key Takeaway"
        "Also, we have some annotation for you to understand table columns"
        "- **Valuation Date:** The date on which the option price is calculated."
        "- **Spot Price (S):** Current price of the underlying asset."
        "- **Strike Price (K):** Exercise price of the option."
        "- **Time to Maturity (T):** Time remaining until option expiration, expressed in years."
        "- **Risk-Free Rate (r):** Annualized risk-free interest rate, used for discounting."
        "- **Volatility (σ):** Annualized standard deviation of the underlying asset’s returns."
        "- **Option Type:** Call or Put."
        "- **Asset Class:** Classification of the underlying asset (e.g., equity, index)."
    )

    user_prompt = (
        "Here is the raw pull and call price tables that should become the option pricing output summary section. The title should have this asset class name."
        "Please refine it as described:\n\n"
        f"{df_call}"
        f"{df_put}"
    )

    # # Using chat.completions; you can swap to Responses API if you prefer
    # completion = client.chat.completions.create(
    #     model="gpt-4.1-mini",
    #     messages=[
    #         {"role": "system", "content": system_prompt},
    #         {"role": "user", "content": user_prompt},
    #     ],
    #     temperature=0.3,
    # )
            

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_prompt),
    ]
    ai_msg = llm.invoke(messages)
    refined_summary1 = ai_msg.content
    
    for block in refined_summary1.split("\n\n"):
        block = block.strip()
        if block:
            doc.add_paragraph(block)   

    # Insert figure
    doc.add_picture(img_stream, width=Inches(6.5))

    # Create table (rows = data + header)
    table = doc.add_table(rows=df.shape[0] + 1, cols=df.shape[1])
    table.style = "Table Grid"

    # Header row
    for col_idx, col_name in enumerate(df.columns):
        cell = table.rows[0].cells[col_idx]
        cell.text = str(col_name)
        cell.paragraphs[0].runs[0].bold = True

    # Data rows
    for row_idx in range(df.shape[0]):
        for col_idx in range(df.shape[1]):
            table.rows[row_idx + 1].cells[col_idx].text = str(df.iat[row_idx, col_idx])


In [127]:
doc.save(final_report_path)

In [95]:
asset="Equity"
doc.add_paragraph("The pricing output of "+asset+ " listed in the below table,")
df = df_validate[df_validate["asset_class"] == asset]
df = df.sort_values("T")
# df = df.dropna()


In [106]:
fig, ax = plt.subplots()
df_call = df[df["option_type"] == "call"]
ax.plot(df_call["T"], df_call["BSM_price"], label = "call")
df_put = df[df["option_type"] == "put"]
ax.plot(df_put["T"], df_put["BSM_price"], label = "put")

ax.set_xlabel("Time to Maturity (T)")
ax.set_ylabel("Option Price (BSM)")
ax.set_title(f"Option Pricing Curve – {asset}")  
ax.legend()

# Save figure to memory (PNG bytes)
img_stream = BytesIO()
fig.savefig(img_stream, format="png", dpi=200, bbox_inches="tight")
plt.close(fig)
img_stream.seek(0)

0

In [107]:
system_prompt = (
    "Please summmarize the option pricing results (pull and call) from the tables with the following topics"
    "1. Overall Data Quality"
    "2. Pricing Level by Asset Class"
    "3. Term Structure (Price vs Maturity)"
    "4. Call vs Put Behavior"
    "5. Model Consistency"
    "6. Key Takeaway"
    "Also, we have some annotation for you to understand table columns"
    "- **Valuation Date:** The date on which the option price is calculated."
    "- **Spot Price (S):** Current price of the underlying asset."
    "- **Strike Price (K):** Exercise price of the option."
    "- **Time to Maturity (T):** Time remaining until option expiration, expressed in years."
    "- **Risk-Free Rate (r):** Annualized risk-free interest rate, used for discounting."
    "- **Volatility (σ):** Annualized standard deviation of the underlying asset’s returns."
    "- **Option Type:** Call or Put."
    "- **Asset Class:** Classification of the underlying asset (e.g., equity, index)."
)

user_prompt = (
    "Here is the raw pull and call price tables that should become the option pricing output summary section. The title should have this asset class name."
    "Please refine it as described:\n\n"
    f"{df_call}"
    f"{df_put}"
)

In [115]:
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]
ai_msg = llm.invoke(messages)
refined_summary1 = ai_msg.content

In [116]:
for block in refined_summary1.split("\n\n"):
    block = block.strip()
    if block:
        doc.add_paragraph(block)   

In [120]:
doc.add_picture(img_stream, width=Inches(6.5))
table = doc.add_table(rows=df.shape[0] + 1, cols=df.shape[1])
table.style = "Table Grid"
for col_idx, col_name in enumerate(df.columns):
        cell = table.rows[0].cells[col_idx]
        cell.text = str(col_name)
        cell.paragraphs[0].runs[0].bold = True

# Data rows
for row_idx in range(df.shape[0]):
    for col_idx in range(df.shape[1]):
        table.rows[row_idx + 1].cells[col_idx].text = str(df.iat[row_idx, col_idx])


### Node Level

In [ ]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
local_tool_folder_path = os.path.join(project_path, "src/bsm_multi_agents/tools")
final_report_path = os.path.join(project_path, "data/output/final_report.docx")

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_folder_path=local_tool_folder_path,
    final_report_path=final_report_path,
)


from bsm_multi_agents.agents import pricing_calculator_node
importlib.reload(pricing_calculator_node)
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)


from bsm_multi_agents.agents.pricing_validator_node import (
    pricing_validator_agent_node,
    pricing_validator_tool_node,
)

from bsm_multi_agents.agents.report_generator_node import (
    report_generator_agent_node,
)

initial_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path,
    local_tool_folder_path=local_tool_folder_path,
    final_report_path=final_report_path,
)

state = pricing_calculator_agent_node(initial_state)
state = pricing_calculator_tool_node(state)
state = pricing_validator_agent_node(state)
state = pricing_validator_tool_node(state)
state = report_generator_agent_node(state)
state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/mcp/server.py',
 'local_tool_paths': ['/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/src/bsm_multi_agents/tools/my_add.py'],
 'final_report_path': '/Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/output/final_report.docx',
 'messages': [SystemMessage(content='You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server. Use the available tools to process the requested data. If you are confident, you can run all tools in parallel.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Input CSV File: /Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv\nOutput Directory: /Users/yifanli/Githu

# Graph

In [8]:
from langgraph.graph import StateGraph, END, START

from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)

from bsm_multi_agents.agents.pricing_validator_node import (
    pricing_validator_agent_node,
    pricing_validator_tool_node,
)

from bsm_multi_agents.agents.report_generator_node import (
    report_generator_agent_node,
)


In [9]:
def should_continue_for_pricing_calculator(state):
    messages = state["messages"]
    if not messages:
        return END
        
    last_msg = messages[-1]
    
    # 1. If it's an Agent message (AIMessage) with tool_calls -> Go to Tool
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "pricing_calculator_tool"
    
    # 2. If it's a Tool Execution message (ToolMessage)
    #    Check for errors to decide if we should retry
    if isinstance(last_msg, ToolMessage):
         if last_msg.content.startswith("Error"):
             # OPTIONAL: Check iteration count to prevent infinite loop
             return "pricing_calculator_agent"
         
         # Success -> Finish
         return "pricing_validator_agent"
    
    # Default
    return "pricing_validator_agent"

def should_continue_for_pricing_validator(state):
    messages = state["messages"]
    if not messages:
        return END
        
    last_msg = messages[-1]
    
    # 1. If it's an Agent message (AIMessage) with tool_calls -> Go to Tool
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "pricing_validator_tool"
    
    # 2. If it's a Tool Execution message (ToolMessage)
    #    Check for errors to decide if we should retry
    if isinstance(last_msg, ToolMessage):
         if last_msg.content.startswith("Error"):
             # OPTIONAL: Check iteration count to prevent infinite loop
             return "pricing_validator_agent"
         
         # Success -> Finish
         return "report_generator_agent"
    
    # Default
    return "report_generator_agent"

In [10]:
graph = StateGraph(WorkflowState)
graph.add_node("pricing_calculator_agent", pricing_calculator_agent_node)
graph.add_node("pricing_calculator_tool", pricing_calculator_tool_node)
graph.add_node("pricing_validator_agent", pricing_validator_agent_node)
graph.add_node("pricing_validator_tool", pricing_validator_tool_node)
graph.add_node("report_generator_agent", report_generator_agent_node)

graph.add_edge(START, "pricing_calculator_agent")
graph.add_edge("pricing_calculator_agent", "pricing_calculator_tool")
graph.add_conditional_edges(
    "pricing_calculator_tool",
    should_continue_for_pricing_calculator,
    {
        "pricing_calculator_agent": "pricing_calculator_agent", # Retry
        "pricing_validator_agent": "pricing_validator_agent" # Success
    }
)
graph.add_edge("pricing_validator_agent", "pricing_validator_tool")
graph.add_conditional_edges(
    "pricing_validator_tool",
    should_continue_for_pricing_validator,
    {
        "pricing_validator_agent": "pricing_validator_agent", # Retry
        "report_generator_agent": "report_generator_agent" # Success
    }
)
graph.add_edge("report_generator_agent", END)
app = graph.compile()

In [11]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
local_tool_paths = [os.path.join(project_path, "src/bsm_multi_agents/tools/my_add.py")]
final_report_path = os.path.join(project_path, "data/output/final_report.docx")

init_state = WorkflowState(
    csv_file_path=csv_file_path,
    output_dir=output_dir,
    server_path=server_path,
    local_tool_paths=local_tool_paths,
    final_report_path=final_report_path,
    # "remaining_steps": 10,
    # "messages": [HumanMessage(content=f"Load CSV from: {file_path}")],
)

final_state = app.invoke(
    init_state,
    config={"configurable": {"thread_id": "run-1"}}
)

In [13]:
final_state

{'messages': [SystemMessage(content='You are a quantitative calculator agent. You have access to tools specifically for Greeks calculation via an MCP server. Use the available tools to process the requested data. If you are confident, you can run all tools in parallel.', additional_kwargs={}, response_metadata={}, id='48e9a4a0-1671-4356-8145-c0ddba941b91'),
  HumanMessage(content='Input CSV File: /Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/input/dummy_options.csv\nOutput Directory: /Users/yifanli/Github/model_doc_automation/TooTwo_mcp/data/cache\n\nPlease calculate the Greeks for the options in the input CSV file. Save the results to the output directory. Ensure you call the calculation tools.', additional_kwargs={}, response_metadata={}, id='e6fd5356-4926-4669-ba04-1453950362c0'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:8b', 'created_at': '2025-12-18T00:31:27.20362Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6216116